In [ ]:
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# --- 1. Inicialização do Spark ---
spark = SparkSession.builder \
    .appName("ProcessamentoSilver") \
    .getOrCreate()

print("Sessão Spark iniciada para Processamento SILVER.")

# --- 2. Variáveis e Schemas ---
# Caminhos locais para ler os dados brutos criados no script anterior
CAMINHO_BRONZE_SUICIDIO = "dados_bronze_suicidio.json"
CAMINHO_BRONZE_DEPRESSAO = "dados_bronze_depressao.json"

# Caminhos do HDFS para salvar a Camada Silver
CAMINHO_SILVER_SUICIDIO = "hdfs://namenode:9000/datalake/silver/suicidio"
CAMINHO_SILVER_DEPRESSAO = "hdfs://namenode:9000/datalake/silver/depressao"

# Schema do dado bruto (manualmente definido para segurança)
schema = StructType([
    StructField("Id", IntegerType(), True),
    StructField("SpatialDim", StringType(), True),
    StructField("TimeDim", IntegerType(), True),
    StructField("Dim1", StringType(), True),
    StructField("NumericValue", DoubleType(), True),
    # ... (outros campos do schema original, mantendo a estrutura)
    StructField("IndicatorCode", StringType(), True),
    StructField("SpatialDimType", StringType(), True),
    StructField("ParentLocationCode", StringType(), True),
    StructField("TimeDimType", StringType(), True),
    StructField("ParentLocation", StringType(), True),
    StructField("Dim1Type", StringType(), True),
    StructField("Dim2Type", StringType(), True),
    StructField("Dim2", StringType(), True),
    StructField("Dim3Type", StringType(), True),
    StructField("Dim3", StringType(), True),
    StructField("DataSourceDimType", StringType(), True),
    StructField("DataSourceDim", StringType(), True),
    StructField("Value", StringType(), True),
    StructField("Low", DoubleType(), True),
    StructField("High", DoubleType(), True),
    StructField("Comments", StringType(), True),
    StructField("Date", StringType(), True),
    StructField("TimeDimensionValue", StringType(), True),
    StructField("TimeDimensionBegin", StringType(), True),
    StructField("TimeDimensionEnd", StringType(), True)
])

# --- 3. Função de Criação e Tratamento do DataFrame (Camada Silver) ---
def criar_df_tratado(caminho_dados, dataset_nome):
    print(f"Lendo dados brutos de {caminho_dados}...")
    try:
        with open(caminho_dados, 'r') as f:
            dados = json.load(f)
    except FileNotFoundError:
        print(f"Erro: Arquivo {caminho_dados} não encontrado. Execute ingestao_bronze.py primeiro.")
        return None

    # Converte a lista de JSONs em RDD e depois em DataFrame
    json_strings = [json.dumps(item) for item in dados]
    rdd = spark.sparkContext.parallelize(json_strings)
    df = spark.read.json(rdd, schema=schema)
    
    # Aplicando transformações (limpeza e organização)
    df_tratado = df.select(
        col("SpatialDim").alias("Pais"),
        col("TimeDim").alias("Ano"),
        col("Dim1").alias("Sexo"),
        col("NumericValue").alias("Valor")
    ).withColumn("dataset_origem", lit(dataset_nome))
    
    return df_tratado

# --- 4. Execução do Processamento e Carga na Camada SILVER ---
df_suicidio = criar_df_tratado(CAMINHO_BRONZE_SUICIDIO, "suicidio")
df_depressao = criar_df_tratado(CAMINHO_BRONZE_DEPRESSAO, "depressao")

if df_suicidio is None or df_depressao is None:
    spark.stop()
    exit()

print("\nSalvando dados tratados (Camada SILVER) no HDFS...")
df_suicidio.write.mode("overwrite").parquet(CAMINHO_SILVER_SUICIDIO)
df_depressao.write.mode("overwrite").parquet(CAMINHO_SILVER_DEPRESSAO)

print("Dados SILVER salvos com sucesso no HDFS!")
df_suicidio.show(5)
df_depressao.show(5)

# --- 5. Finalização ---
spark.stop()
print("Sessão Spark de Processamento SILVER finalizada.")

'pip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


ModuleNotFoundError: No module named 'pyspark'

: 